In [1]:
# Welcome to your new notebook
# Type here in the cell editor to add code!
import requests, pandas as pd
from datetime import datetime
from time import sleep
import pytz

API_KEY = "492aa1d288cfdf972a104e0a55c35b68"
BASE_URL_FORECAST = "https://api.openweathermap.org/data/2.5/forecast"

# Clean city list (strip spaces)
city_list = ["Delhi","Mumbai","Bangalore","Chennai","Kolkata","Hyderabad","Pune",
             "Jaipur","Lucknow","Surat","Kanpur","Nagpur","Indore","Bhopal",
             "Patna","Vadodara","Ludhiana","Agra","Varanasi"]

session = requests.Session()
session.headers.update({"User-Agent":"weather-crawler/1.0"})
timeout = 15  # seconds

all_rows = []
india_tz = pytz.timezone('Asia/Kolkata')
now_stamp = datetime.now(india_tz).strftime("%Y-%m-%d %H:%M:%S")

for city in city_list:
    params = {"q": city, "appid": API_KEY, "units": "metric"}
    try:
        r = session.get(BASE_URL_FORECAST, params=params, timeout=timeout)
        data = r.json()
        if r.status_code != 200:
            print(f"[WARN] Forecast failed for {city}: {data}")
            continue

        # data["list"] = list of 3-hour forecast steps (up to 40 items)
        # attach city/coord meta once
        city_meta = data.get("city", {})
        for item in data.get("list", []):
            main = item.get("main", {})
            wind = item.get("wind", {})
            clouds = item.get("clouds", {})
            weather0 = (item.get("weather") or [{}])[0]
            rain = item.get("rain", {})
            snow = item.get("snow", {})

            all_rows.append({
                "crawl_time": now_stamp,               # when we crawled
                "city": city_meta.get("name", city),
                "city_lat": city_meta.get("coord",{}).get("lat"),
                "city_lon": city_meta.get("coord",{}).get("lon"),

                "forecast_time": item.get("dt_txt"),   # 3-hour forecast timestamp (local)
                "temp_c": main.get("temp"),
                "feels_like_c": main.get("feels_like"),
                "temp_min_c": main.get("temp_min"),
                "temp_max_c": main.get("temp_max"),
                "pressure_hpa": main.get("pressure"),
                "humidity_pct": main.get("humidity"),
                "visibility_m": item.get("visibility"),

                "wind_speed_ms": wind.get("speed"),
                "wind_deg": wind.get("deg"),
                "cloudiness_pct": clouds.get("all"),

                "weather_main": weather0.get("main"),
                "weather_desc": weather0.get("description"),

                "pop": item.get("pop"),               # probability of precipitation
                "rain_3h_mm": rain.get("3h"),
                "snow_3h_mm": snow.get("3h"),
            })

        # be gentle with the API (especially on free tier)
        sleep(0.3)

    except Exception as e:
        print(f"[ERR] City {city}: {e}")

df_fc = pd.DataFrame(all_rows)

# Optional: quick glance
display(df_fc.head(10))


StatementMeta(, 4bb842a9-dc1d-4173-8fb0-69635de83b03, 3, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, bda51d30-3647-4d91-891b-8e796140590e)

In [2]:
import pandas as pd
from datetime import datetime
import pytz

# all_weather_data is your list of dicts
df_weather = pd.DataFrame(all_rows)

# today's date for filename
india_tz = pytz.timezone('Asia/Kolkata')
today_str = datetime.now(india_tz).strftime("%Y-%m-%d %H:%M:%S")

# save to Lakehouse Files with date in filename
file_path = f"/lakehouse/default/Files/forecast_data_bronze/weather_forecast {today_str}.csv"
df_weather.to_csv(file_path, index=False)

print(f"Saved: {file_path}")

StatementMeta(, 4bb842a9-dc1d-4173-8fb0-69635de83b03, 4, Finished, Available, Finished)

Saved: /lakehouse/default/Files/forecast_data_bronze/weather_forecast 2025-08-28 09:31:46.csv
